# Feature Engineering
다이소 뷰티 제품 데이터 파생변수 생성

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 데이터 로드
products_df = pd.read_parquet('./data/products.parquet')
reviews_df = pd.read_parquet('./data/reviews.parquet')

print("데이터 로드 완료")
print(f"제품: {len(products_df):,}개")
print(f"리뷰: {len(reviews_df):,}건")
print(f"\n제품 컬럼: {products_df.columns.tolist()}")
print(f"\n리뷰 컬럼: {reviews_df.columns.tolist()}")

데이터 로드 완료
제품: 438개
리뷰: 85,520건

제품 컬럼: ['product_code', 'category_home', 'category_1', 'category_2', 'brand', 'name', 'price', 'country', 'likes', 'shares', 'url', 'can_할랄인증', 'can_비건', 'certifications']

리뷰 컬럼: ['product_code', 'date', 'user_masked', 'user', 'rating', 'text', 'image_count']


## A. 상품 관점: "진짜 인기 상품" 찾기

In [2]:
# 제품별 리뷰 수 계산
review_counts = reviews_df.groupby('product_code').size().reset_index(name='review_count')

# products_df에 병합
products_df = products_df.merge(review_counts, on='product_code', how='left')
products_df['review_count'] = products_df['review_count'].fillna(0).astype(int)

print(f"리뷰 수 추가 완료")
print(f"리뷰가 있는 제품: {(products_df['review_count'] > 0).sum()}개")
print(f"리뷰가 없는 제품: {(products_df['review_count'] == 0).sum()}개")

리뷰 수 추가 완료
리뷰가 있는 제품: 433개
리뷰가 없는 제품: 5개


### 가중치 결정을 위한 데이터 분석

In [3]:
# 상관관계와 분별력 분석
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

print("="*80)
print("Engagement Score 가중치 결정을 위한 데이터 분석")
print("="*80)

# 1. 기본 통계량
print("\n[1] 기본 통계량")
print("-"*80)
stats = products_df[['likes', 'shares', 'review_count']].describe()
print(stats)

Engagement Score 가중치 결정을 위한 데이터 분석

[1] 기본 통계량
--------------------------------------------------------------------------------
             likes      shares  review_count
count   438.000000  438.000000    438.000000
mean   1818.148402   25.105023    195.251142
std    2120.030652   32.998550    275.708467
min       0.000000    0.000000      0.000000
25%     530.250000    6.000000     66.250000
50%    1061.500000   13.000000    121.000000
75%    2085.500000   30.000000    217.000000
max    9999.000000  236.000000   3595.000000


In [4]:
# 2. 변동계수 (Coefficient of Variation) = std / mean
# 값이 클수록 분산이 크고 분별력이 높음
print("\n변동계수 - 분별력 지표")
print("-"*80)
cv_likes = products_df['likes'].std() / products_df['likes'].mean()
cv_shares = products_df['shares'].std() / products_df['shares'].mean()
cv_reviews = products_df['review_count'].std() / products_df['review_count'].mean()

print(f"likes:        CV = {cv_likes:.3f}")
print(f"shares:       CV = {cv_shares:.3f}")
print(f"review_count: CV = {cv_reviews:.3f}")
print(f"\n분별력 순위: ", end="")
cv_dict = {'likes': cv_likes, 'shares': cv_shares, 'review_count': cv_reviews}
sorted_cv = sorted(cv_dict.items(), key=lambda x: x[1], reverse=True)
print(" > ".join([f"{k} ({v:.3f})" for k, v in sorted_cv]))


변동계수 - 분별력 지표
--------------------------------------------------------------------------------
likes:        CV = 1.166
shares:       CV = 1.314
review_count: CV = 1.412

분별력 순위: review_count (1.412) > shares (1.314) > likes (1.166)


In [5]:
# 3. 정규화 후 분산
# 값이 클수록 분별력이 낮음
print("\n정규화 후 분산")
print("-"*80)
scaler = MinMaxScaler()
normalized = scaler.fit_transform(products_df[['likes', 'shares', 'review_count']])
normalized_df = pd.DataFrame(normalized, columns=['likes_norm', 'shares_norm', 'reviews_norm'])

norm_var_likes = normalized_df['likes_norm'].var()
norm_var_shares = normalized_df['shares_norm'].var()
norm_var_reviews = normalized_df['reviews_norm'].var()

print(f"likes:        {norm_var_likes:.4f}")
print(f"shares:       {norm_var_shares:.4f}")
print(f"review_count: {norm_var_reviews:.4f}")


정규화 후 분산
--------------------------------------------------------------------------------
likes:        0.0450
shares:       0.0196
review_count: 0.0059


In [6]:
# 4. 상관관계 분석
print("\n상관관계 매트릭스")
print("-"*80)
corr = products_df[['likes', 'shares', 'review_count']].corr()
print(corr)


상관관계 매트릭스
--------------------------------------------------------------------------------
                 likes    shares  review_count
likes         1.000000  0.759839      0.776910
shares        0.759839  1.000000      0.621563
review_count  0.776910  0.621563      1.000000


In [7]:
# A-1. Engagement Score (실질적 인기도)
# Score = w1*likes + w2*shares + w3*review_count
# 가중치: review_count(0.55) > shares(0.30) > likes(0.15)
# 분별력이 높은 변수에 높은 가중치 부여
w1, w2, w3 = 0.15, 0.30, 0.55  # likes, shares, review_count

products_df['engagement_score'] = (
    w1 * products_df['likes'] + 
    w2 * products_df['shares'] + 
    w3 * products_df['review_count']
)

print("Engagement Score 생성 완료")
print(f"가중치: likes={w1}, shares={w2}, review_count={w3}")
print(f"평균: {products_df['engagement_score'].mean():.2f}")
print(f"최대: {products_df['engagement_score'].max():.2f}")
print("\nTOP 5 인기 제품:")
print(products_df.nlargest(5, 'engagement_score')[['name', 'brand', 'likes', 'shares', 'review_count', 'engagement_score']])

Engagement Score 생성 완료
가중치: likes=0.15, shares=0.3, review_count=0.55
평균: 387.64
최대: 3547.90

TOP 5 인기 제품:
                              name brand  likes  shares  review_count  \
134  입큰 퍼스널 톤 코렉팅 블러 팩트 5.5 g(라벤더)    입큰   9999     236          3595   
141         클라뷰 밸런싱 포어밤 프라이머 10 ml   클라뷰   9999     181          1610   
111           태그 슬림브로우펜슬(2호_애쉬브라운)    태그   9999     146          1475   
235       손앤박 아티 스프레드 컬러 밤(03 멜로우)   손앤박   9999      52          1515   
1            더봄 애교살＆트임 펜슬 (음영 브라운)    더봄   9999     184          1242   

     engagement_score  
134           3547.90  
141           2439.65  
111           2354.90  
235           2348.70  
1             2238.15  


In [8]:
# A-2-1. price_tier (절대적 가격 티어)
def classify_daiso_price(price):
    if price <= 1500:
        return 'Easy Pick (Low)'
    elif price <= 3000:
        return 'Standard (Mid)'
    else: # 5000원 등
        return 'Premium (High)'

products_df['price_tier'] = products_df['price'].apply(classify_daiso_price)

print("price_tier 생성 완료")
print("\n가격대별 분포:")
print(products_df['price_tier'].value_counts())
print("\n카테고리별 가격대 분포:")
print(pd.crosstab(products_df['category_2'], products_df['price_tier']))

price_tier 생성 완료

가격대별 분포:
price_tier
Standard (Mid)     266
Premium (High)     151
Easy Pick (Low)     21
Name: count, dtype: int64

카테고리별 가격대 분포:
price_tier  Easy Pick (Low)  Premium (High)  Standard (Mid)
category_2                                                 
남성메이크업                    0               2               0
남성스킨케어                    1              18               7
남성용면도기                    6               8              16
남성향수                      0               0               6
립메이크업                     2              24             106
베이스메이크업                   0              70              35
아이메이크업                   12              27              89
클렌징/쉐이빙                   0               2               7


In [9]:
# A-2-2. price_position (카테고리 대비 가격 지수)
# 1. 소분류별 평균 가격 계산
category_avg_price = products_df.groupby('category_2')['price'].transform('mean')

# 2. 비율 계산 (1.0 = 평균, 1.5 = 평균보다 1.5배 비쌈)
products_df['relative_price_ratio'] = products_df['price'] / category_avg_price

# 3. 구간화
def categorize_relative_price(ratio):
    if ratio < 0.8: return 'Cheaper than Avg'
    elif ratio > 1.2: return 'More Expensive than Avg'
    else: return 'Average'

products_df['price_position'] = products_df['relative_price_ratio'].apply(categorize_relative_price)

print("Price Position 생성완료\n")
print(products_df['price_position'].value_counts(normalize=True).mul(100).round(1))
print("\n카테고리별 Price Position 분포")
print(pd.crosstab(products_df['category_2'],products_df['price_position'],normalize='index').mul(100).round(1).head(5))


Price Position 생성완료

price_position
Average                    65.3
Cheaper than Avg           20.8
More Expensive than Avg    13.9
Name: proportion, dtype: float64

카테고리별 Price Position 분포
price_position  Average  Cheaper than Avg  More Expensive than Avg
category_2                                                        
남성메이크업            100.0               0.0                      0.0
남성스킨케어             69.2              30.8                      0.0
남성용면도기             30.0              43.3                     26.7
남성향수              100.0               0.0                      0.0
립메이크업              80.3               1.5                     18.2


In [10]:
# A-2-3. price_tier (가성비 지수)
# 숫자가 너무 작아지는 것을 방지하기 위해 * 1000
products_df['cp_index'] = (products_df['engagement_score'] / products_df['price']) * 1000

# 상위 10% 
quantile_90 = products_df['cp_index'].quantile(0.9)
products_df['is_god_sung_bi'] = products_df['cp_index'] >= quantile_90

print("Price_tier 생성 완료")
print("\n가성비(Top 10%) 선정 상품의 가격대 분포")
god_sung_bi_items = products_df[products_df['is_god_sung_bi'] == True]
print(god_sung_bi_items['price'].value_counts(normalize=True).sort_index().mul(100).round(1))
print("\n전체 상품 가격대 분포")
print(products_df['price'].value_counts(normalize=True).sort_index().mul(100).round(1))
print("\n가성비 아이템 예시 (Top 5)")
display(god_sung_bi_items[['category_1', 'name', 'price', 'cp_index']].sort_values(by='cp_index', ascending=False).head(5))

Price_tier 생성 완료

가성비(Top 10%) 선정 상품의 가격대 분포
price
2000     6.8
3000    75.0
5000    18.2
Name: proportion, dtype: float64

전체 상품 가격대 분포
price
500      0.2
1000     4.6
2000     6.8
3000    53.9
5000    34.5
Name: proportion, dtype: float64

가성비 아이템 예시 (Top 5)


,category_1,name,price,cp_index
134,메이크업,입큰 퍼스널 톤 코렉팅 블러 팩트 5.5 g(라벤더),3000,1182.633333
1,메이크업,더봄 애교살＆트임 펜슬 (음영 브라운),2000,1119.075000
7,메이크업,더봄 애교살＆트임 펜슬(볼륨베이지),2000,1035.375000
141,메이크업,클라뷰 밸런싱 포어밤 프라이머 10 ml,3000,813.216667
111,메이크업,태그 슬림브로우펜슬(2호_애쉬브라운),3000,784.966667


In [11]:
# A-3. Review Density (리뷰 밀도)
# Review Count / Likes (likes 대비 리뷰 작성 비율)
products_df['review_density'] = products_df.apply(
    lambda x: x['review_count'] / x['likes'] if x['likes'] > 0 else 0,
    axis=1
)

print("Review Density 생성 완료")
print(f"평균 리뷰 밀도: {products_df['review_density'].mean():.4f}")
print(f"최대 리뷰 밀도: {products_df['review_density'].max():.4f}")
print("\n리뷰 밀도가 높은 제품 TOP 5")
print(products_df.nlargest(5, 'review_density')[['name', 'brand', 'likes', 'review_count', 'review_density']])

Review Density 생성 완료
평균 리뷰 밀도: 0.1427
최대 리뷰 밀도: 1.4727

리뷰 밀도가 높은 제품 TOP 5
                            name    brand  likes  review_count  review_density
353  [02 애플팝] 코드글로컬러 프루티 볼륨 립글로스      [02     55            81        1.472727
173      [04 커버 베이지] 드롭비 컬러즈 컨실팟  드롭비 컬러즈    188           164        0.872340
215     [02 피치] 코드글로컬러 컨실레놀 10 g      [02    115            96        0.834783
213     [01 퓨어] 코드글로컬러 컨실레놀 10 g      [01    102            77        0.754902
425           쉬크 이그젝타 3중날 면도기 4P       쉬크     45            32        0.711111


## B. 리뷰 관점

In [12]:
# B-1. Review Length (리뷰 길이)
reviews_df['review_length'] = reviews_df['text'].fillna('').str.len()

print("[B-1] Review Length 생성 완료")
print(f"평균 리뷰 길이: {reviews_df['review_length'].mean():.2f}자")
print(f"중간값: {reviews_df['review_length'].median():.0f}자")
print(f"최대: {reviews_df['review_length'].max()}자")

# 리뷰 길이 구간별 분류
def classify_review_length(length):
    if length < 20:
        return 'Very Short'
    elif length < 50:
        return 'Short'
    elif length < 100:
        return 'Medium'
    else:
        return 'Long'

reviews_df['review_length_category'] = reviews_df['review_length'].apply(classify_review_length)
print("\n리뷰 길이 분포:")
print(reviews_df['review_length_category'].value_counts())

[B-1] Review Length 생성 완료
평균 리뷰 길이: 38.14자
중간값: 24자
최대: 996자

리뷰 길이 분포:
review_length_category
Short         36741
Very Short    31561
Medium        12201
Long           5017
Name: count, dtype: int64


## C. 유저 관점

In [13]:
# C-0. 재구매 명시 여부 (텍스트가 '재구매'로 시작)
reviews_df['is_reorder'] = reviews_df['text'].fillna('').str.strip().str.startswith('재구매')

print("is_reorder 생성 완료")
reorder_count = reviews_df['is_reorder'].sum()
print(f"\n'재구매'로 시작하는 리뷰: {reorder_count:,}건 ({reorder_count/len(reviews_df)*100:.2f}%)")

is_reorder 생성 완료

'재구매'로 시작하는 리뷰: 15,580건 (18.22%)


In [14]:
# C-1. User Activity Level (유저 활동 등급)
user_review_counts = reviews_df.groupby('user').size().reset_index(name='user_total_reviews')
reviews_df = reviews_df.merge(user_review_counts, on='user', how='left')

def classify_user_activity(count):
    if count <= 1:
        return 'Newbie'
    elif count <= 5:
        return 'Junior'
    elif count <= 20:
        return 'Regular'
    else:
        return 'VIP'

reviews_df['user_activity_level'] = reviews_df['user_total_reviews'].apply(classify_user_activity)

print("User Activity Level 생성 완료")
print("\n유저 등급 분포:")
print(reviews_df['user_activity_level'].value_counts())
print(f"\n고유 user_masked 수: {reviews_df['user_masked'].nunique():,}명")
print(f"\n고유 user 수: {reviews_df['user'].nunique():,}명")

User Activity Level 생성 완료

유저 등급 분포:
user_activity_level
Regular    38232
VIP        28354
Junior     16258
Newbie      2676
Name: count, dtype: int64

고유 user_masked 수: 15,080명

고유 user 수: 12,335명


In [15]:
# C-2. User Average Rating (재구매 고객의 평균 평점)

reorder_reviews_only = reviews_df[reviews_df['is_reorder'] == True].copy()

print("User Average Rating (재구매 고객 기준) 생성")
print(f"전체 리뷰: {len(reviews_df):,}건")
print(f"재구매 리뷰: {len(reorder_reviews_only):,}건")

if len(reorder_reviews_only) > 0:
    user_avg_rating_reorder = reorder_reviews_only.groupby('user')['rating'].mean().reset_index(name='user_avg_rating_reorder')
    
    # 전체 리뷰에 병합 (재구매 경험 없는 유저는 NaN)
    reviews_df = reviews_df.merge(user_avg_rating_reorder, on='user', how='left')
    
    # 평점 부여 성향 분류
    def classify_rating_tendency(avg_rating):
        if pd.isna(avg_rating):
            return 'No Reorder'
        elif avg_rating >= 4.8:
            return 'Always Positive'
        elif avg_rating >= 4.0:
            return 'Mostly Positive'
        elif avg_rating >= 3.0:
            return 'Balanced'
        else:
            return 'Critical'
    
    reviews_df['user_rating_tendency'] = reviews_df['user_avg_rating_reorder'].apply(classify_rating_tendency)
    
    print("\nUser Average Rating 생성 완료")
    print(f"재구매 경험 있는 유저: {user_avg_rating_reorder['user'].nunique():,}명")
    print(f"재구매 리뷰 평균 평점: {user_avg_rating_reorder['user_avg_rating_reorder'].mean():.3f}")
    
    print("\n유저 평점 성향 분포:")
    print(reviews_df['user_rating_tendency'].value_counts())
    
else:
    reviews_df['user_avg_rating_reorder'] = np.nan
    reviews_df['user_rating_tendency'] = 'No Reorder'
    print("\n재구매 리뷰가 없습니다.")

User Average Rating (재구매 고객 기준) 생성
전체 리뷰: 85,520건
재구매 리뷰: 15,580건

User Average Rating 생성 완료
재구매 경험 있는 유저: 5,142명
재구매 리뷰 평균 평점: 4.752

유저 평점 성향 분포:
user_rating_tendency
Always Positive    42736
No Reorder         25162
Mostly Positive    15246
Balanced            1939
Critical             437
Name: count, dtype: int64


In [16]:
# C-3. Repurchase Indicator
# is_reorder=True인 리뷰들만 기준으로 같은 브랜드/카테고리 재구매 확인

# 제품 정보 병합 (brand, category 정보 가져오기)
reviews_with_product = reviews_df.merge(
    products_df[['product_code', 'brand', 'category_2']], 
    on='product_code', 
    how='left'
)

# 날짜 변환
reviews_with_product['date'] = pd.to_datetime(reviews_with_product['date'])

# 유저별로 정렬
reviews_with_product = reviews_with_product.sort_values(['user', 'date'])

# is_reorder=True인 리뷰들만 필터링
reorder_reviews = reviews_with_product[reviews_with_product['is_reorder'] == True].copy()

print("Repurchase Indicator 생성 완료")
print(f"'재구매' 명시 리뷰: {len(reorder_reviews):,}건")

# 같은 브랜드 재구매 여부
if len(reorder_reviews) > 0:
    reorder_reviews['is_brand_repurchase'] = (
        reorder_reviews.groupby('user')['brand']
        .transform(lambda x: x.duplicated(keep=False))
    ).astype(int)
    
    # 같은 카테고리 재구매 여부
    reorder_reviews['is_category_repurchase'] = (
        reorder_reviews.groupby('user')['category_2']
        .transform(lambda x: x.duplicated(keep=False))
    ).astype(int)
    
    # 원본 reviews_df에 병합 (기본값은 0)
    reviews_df['is_brand_repurchase'] = 0
    reviews_df['is_category_repurchase'] = 0
    
    # is_reorder=True인 것들만 업데이트
    reviews_df.loc[reviews_df['is_reorder'] == True, 'is_brand_repurchase'] = \
        reorder_reviews['is_brand_repurchase'].values
    reviews_df.loc[reviews_df['is_reorder'] == True, 'is_category_repurchase'] = \
        reorder_reviews['is_category_repurchase'].values
    
    print(f"\n'재구매' 리뷰 중:")
    print(f"  - 같은 브랜드 재구매: {reviews_df['is_brand_repurchase'].sum():,}건")
    print(f"  - 같은 카테고리 재구매: {reviews_df['is_category_repurchase'].sum():,}건")
else:
    reviews_df['is_brand_repurchase'] = 0
    reviews_df['is_category_repurchase'] = 0
    print("\n'재구매' 명시 리뷰가 없습니다.")

Repurchase Indicator 생성 완료
'재구매' 명시 리뷰: 15,580건

'재구매' 리뷰 중:
  - 같은 브랜드 재구매: 9,404건
  - 같은 카테고리 재구매: 11,695건


## D. 시계열(Time) 관점

In [17]:
# D-1. Seasonality (계절성) - 년도별 분석
reviews_df['date'] = pd.to_datetime(reviews_df['date'])
reviews_df['year'] = reviews_df['date'].dt.year
reviews_df['month'] = reviews_df['date'].dt.month
reviews_df['day_of_week'] = reviews_df['date'].dt.dayofweek  # 0=월요일, 6=일요일
reviews_df['day_name'] = reviews_df['date'].dt.day_name()

# 계절 분류
def classify_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

reviews_df['season'] = reviews_df['month'].apply(classify_season)

print("Seasonality 생성 완료")
print("="*80)

# 1. 년도별 리뷰 수
print("\n[1] 년도별 리뷰 수")
print("-"*80)
year_counts = reviews_df['year'].value_counts().sort_index()
print(year_counts)

# 2. 년도별 월별 리뷰 수
print("\n[2] 년도별 월별 리뷰 수")
print("-"*80)
year_month_counts = reviews_df.groupby(['year', 'month']).size().reset_index(name='count')
for year in sorted(reviews_df['year'].unique()):
    year_data = year_month_counts[year_month_counts['year'] == year]
    if len(year_data) > 0:
        print(f"\n{year}년:")
        for _, row in year_data.iterrows():
            print(f"  {int(row['month']):2d}월: {int(row['count']):,}건")

# 3. 년도별 계절별 리뷰 수
print("\n[3] 년도별 계절별 리뷰 수")
print("-"*80)
year_season_counts = reviews_df.groupby(['year', 'season']).size().reset_index(name='count')
for year in sorted(reviews_df['year'].unique()):
    year_data = year_season_counts[year_season_counts['year'] == year]
    if len(year_data) > 0:
        print(f"\n{year}년:")
        season_order = ['Spring', 'Summer', 'Fall', 'Winter']
        for season in season_order:
            season_data = year_data[year_data['season'] == season]
            if len(season_data) > 0:
                count = int(season_data['count'].values[0])
                print(f"  {season:8s}: {count:,}건")

# 4. 전체 요일별 분포
print("\n[4] 요일별 리뷰 수 (전체)")
print("-"*80)
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_counts = reviews_df['day_name'].value_counts()
for day in day_order:
    if day in day_counts.index:
        print(f"{day:10s}: {day_counts[day]:,}건")

Seasonality 생성 완료

[1] 년도별 리뷰 수
--------------------------------------------------------------------------------
year
2020        1
2021       36
2022        6
2023      103
2024    22359
2025    56825
2026     6190
Name: count, dtype: int64

[2] 년도별 월별 리뷰 수
--------------------------------------------------------------------------------

2020년:
  10월: 1건

2021년:
   1월: 1건
   2월: 3건
   3월: 4건
   4월: 5건
   5월: 5건
   6월: 5건
   8월: 1건
   9월: 2건
  11월: 1건
  12월: 9건

2022년:
   2월: 1건
   4월: 1건
   6월: 1건
   7월: 1건
   8월: 1건
  10월: 1건

2023년:
   1월: 1건
   2월: 2건
   3월: 1건
   6월: 1건
   8월: 2건
   9월: 1건
  10월: 1건
  11월: 4건
  12월: 90건

2024년:
   1월: 507건
   2월: 739건
   3월: 1,211건
   4월: 1,190건
   5월: 1,531건
   6월: 1,812건
   7월: 2,539건
   8월: 2,067건
   9월: 1,728건
  10월: 2,153건
  11월: 2,799건
  12월: 4,083건

2025년:
   1월: 4,766건
   2월: 4,236건
   3월: 5,747건
   4월: 4,322건
   5월: 4,571건
   6월: 4,040건
   7월: 4,920건
   8월: 4,741건
   9월: 5,659건
  10월: 4,560건
  11월: 4,560건
  12월: 4,703건

2026년:
   1월: 5,65

In [18]:
# D-2. Days Since First Review (출시 후 경과일)
# 제품별 첫 리뷰 날짜
first_review_dates = reviews_df.groupby('product_code')['date'].min().reset_index(name='first_review_date')
reviews_df = reviews_df.merge(first_review_dates, on='product_code', how='left')

# 경과일 계산
reviews_df['days_since_first_review'] = (
    reviews_df['date'] - reviews_df['first_review_date']
).dt.days

# 신상품 여부 (첫 리뷰 후 30일 이내)
reviews_df['is_new_product'] = (reviews_df['days_since_first_review'] <= 30).astype(int)

print("Days Since First Review 생성 완료")
print(f"평균 경과일: {reviews_df['days_since_first_review'].mean():.1f}일")
print(f"최대 경과일: {reviews_df['days_since_first_review'].max()}일")
print(f"\n신상품(30일 이내) 리뷰: {reviews_df['is_new_product'].sum():,}건 ({reviews_df['is_new_product'].sum()/len(reviews_df)*100:.2f}%)")

Days Since First Review 생성 완료
평균 경과일: 292.4일
최대 경과일: 1929일

신상품(30일 이내) 리뷰: 14,708건 (17.20%)


## E. 프로모션 관점

In [19]:
promo_df = pd.read_csv('data/promotion.csv')
promo_df['date'] = pd.to_datetime(promo_df['date'])

print("Promotion 파생변수 생성")
print(f"프로모션 데이터: {len(promo_df):,}건")
print(f"프로모션 기간: {promo_df['date'].min()} ~ {promo_df['date'].max()}")
print(f"\nevent_type 분포:")
print(promo_df['event_type'].value_counts())

Promotion 파생변수 생성
프로모션 데이터: 118건
프로모션 기간: 2024-09-11 00:00:00 ~ 2026-02-08 00:00:00

event_type 분포:
event_type
리뷰이벤트    116
구매이벤트      2
Name: count, dtype: int64


In [20]:
# E-3. 프로모션 기간 여부 확인
# 리뷰 날짜가 프로모션 날짜와 일치하거나 직후 7일 이내인 경우
from datetime import timedelta

# promo_type_category가 없으면 생성
if 'promo_type_category' not in promo_df.columns:
    def categorize_promo_type(event_type, description):
        event_type = str(event_type).lower()
        description = str(description).lower()
        if '리뷰' in event_type or 'review' in event_type:
            return '리뷰이벤트'
        elif '할인' in description or 'sale' in description:
            return '할인'
        elif '증정' in description or 'gift' in description:
            return '증정'
        elif '구매' in event_type:
            return '구매이벤트'
        else:
            return '기타'
    promo_df['promo_type_category'] = promo_df.apply(
        lambda x: categorize_promo_type(x['event_type'], x['description']), axis=1
    )

# 제품 정보(brand) 병합
reviews_with_brand = reviews_df.merge(
    products_df[['product_code', 'brand']],
    on='product_code',
    how='left'
)

def check_promo_period(review_row, promo_df):
    """리뷰가 프로모션 기간 중/직후에 작성되었는지 확인"""
    review_date = review_row['date']
    review_brand = review_row['brand']
    
    # 브랜드별 프로모션 또는 전체 프로모션('-')과 비교
    # 프로모션 날짜 ± 7일 범위
    if pd.isna(review_brand) or review_brand == '':
        matching_promos = promo_df[
            (promo_df['date'] <= review_date) &
            (promo_df['date'] >= review_date - timedelta(days=7))
        ]
    else:
        matching_promos = promo_df[
            ((promo_df['brand'] == review_brand) | (promo_df['brand'] == '-')) &
            (promo_df['date'] <= review_date) &
            (promo_df['date'] >= review_date - timedelta(days=7))
        ]
    
    if len(matching_promos) > 0:
        closest_promo = matching_promos.iloc[0]
        return True, closest_promo['promo_type_category']
    else:
        return False, '프로모션 기간 아님'

# 벡터화된 방법으로 최적화
results = []
for idx, row in reviews_with_brand.iterrows():
    if idx % 10000 == 0 and idx > 0:
        print(f"  처리 중: {idx:,}/{len(reviews_with_brand):,} ({idx/len(reviews_with_brand)*100:.1f}%)")
    is_promo, promo_type = check_promo_period(row, promo_df)
    results.append((is_promo, promo_type))

# 결과를 reviews_df에 추가
reviews_df['is_during_promo'] = [r[0] for r in results]
reviews_df['promo_type_category'] = [r[1] for r in results]

print(f"\n완료!")

  처리 중: 10,000/85,520 (11.7%)
  처리 중: 20,000/85,520 (23.4%)
  처리 중: 30,000/85,520 (35.1%)
  처리 중: 40,000/85,520 (46.8%)
  처리 중: 50,000/85,520 (58.5%)
  처리 중: 60,000/85,520 (70.2%)
  처리 중: 70,000/85,520 (81.9%)
  처리 중: 80,000/85,520 (93.5%)

완료!


In [21]:
# 결과 확인
promo_reviews = reviews_df['is_during_promo'].sum()
print("[E-4] 프로모션 파생변수 결과")
print("="*80)
print(f"\n프로모션 기간 중/직후 리뷰: {promo_reviews:,}건 ({promo_reviews/len(reviews_df)*100:.2f}%)")
print(f"\n프로모션 유형별 리뷰 수:")
print(reviews_df['promo_type_category'].value_counts())
print(f"\n리뷰이벤트 기간 리뷰 비율:")
review_event_count = (reviews_df['promo_type_category'] == '리뷰이벤트').sum()
print(f"  {review_event_count:,}건 ({review_event_count/len(reviews_df)*100:.2f}%)")


[E-4] 프로모션 파생변수 결과

프로모션 기간 중/직후 리뷰: 3,383건 (3.96%)

프로모션 유형별 리뷰 수:
promo_type_category
프로모션 기간 아님    82137
리뷰이벤트          3383
Name: count, dtype: int64

리뷰이벤트 기간 리뷰 비율:
  3,383건 (3.96%)


## 최종 데이터 저장

In [22]:
# Parquet으로 저장
products_df.to_parquet('./data/products_with_features.parquet', index=False)
reviews_df.to_parquet('./data/reviews_with_features.parquet', index=False)

print("="*80)
print("Feature Engineering 완료!")
print("="*80)
print(f"\nproducts_with_features.parquet 저장 완료")
print(f"  - 제품 수: {len(products_df):,}개")
print(f"  - 컬럼 수: {len(products_df.columns)}개")
print(f"  - 추가된 컬럼: review_count, engagement_score, price_percentile, price_status, review_density")

print(f"\nreviews_with_features.parquet 저장 완료")
print(f"  - 리뷰 수: {len(reviews_df):,}건")
print(f"  - 컬럼 수: {len(reviews_df.columns)}개")
print(f"  - 추가된 컬럼: review_length, sentiment_score, 키워드 태그들, user_activity_level, user_avg_rating, 재구매 지표, 시계열 변수들")

print("\n생성된 파일:")
print("  - products_with_features.parquet")
print("  - reviews_with_features.parquet")

Feature Engineering 완료!

products_with_features.parquet 저장 완료
  - 제품 수: 438개
  - 컬럼 수: 22개
  - 추가된 컬럼: review_count, engagement_score, price_percentile, price_status, review_density

reviews_with_features.parquet 저장 완료
  - 리뷰 수: 85,520건
  - 컬럼 수: 26개
  - 추가된 컬럼: review_length, sentiment_score, 키워드 태그들, user_activity_level, user_avg_rating, 재구매 지표, 시계열 변수들

생성된 파일:
  - products_with_features.parquet
  - reviews_with_features.parquet


In [23]:
# 최종 컬럼 목록 확인
print("\n[Products DataFrame]")
for i, col in enumerate(products_df.columns, 1):
    print(f"{i:2d}. {col}")

print("\n[Reviews DataFrame]")
for i, col in enumerate(reviews_df.columns, 1):
    print(f"{i:2d}. {col}")


[Products DataFrame]
 1. product_code
 2. category_home
 3. category_1
 4. category_2
 5. brand
 6. name
 7. price
 8. country
 9. likes
10. shares
11. url
12. can_할랄인증
13. can_비건
14. certifications
15. review_count
16. engagement_score
17. price_tier
18. relative_price_ratio
19. price_position
20. cp_index
21. is_god_sung_bi
22. review_density

[Reviews DataFrame]
 1. product_code
 2. date
 3. user_masked
 4. user
 5. rating
 6. text
 7. image_count
 8. review_length
 9. review_length_category
10. is_reorder
11. user_total_reviews
12. user_activity_level
13. user_avg_rating_reorder
14. user_rating_tendency
15. is_brand_repurchase
16. is_category_repurchase
17. year
18. month
19. day_of_week
20. day_name
21. season
22. first_review_date
23. days_since_first_review
24. is_new_product
25. is_during_promo
26. promo_type_category
